In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import opendatasets as od
import tensorflow as tf
import cv2
from tqdm import *

In [2]:
dataset_directory = "C:\\Users\\amith\\Documents\\Datasets\\kaggle\\kaggle".replace("\\","/")
csv_directory = "C:\\Users\\amith\\Documents\\Datasets".replace("\\","/")

In [3]:
train_val_age = pd.read_csv(csv_directory + "/train_age.csv").sort_values(by="imageId")
train_val_gender = pd.read_csv(csv_directory + "/train_gender.csv").sort_values(by="imageId")

id = train_val_age["imageId"].values.tolist()
train_val_files = pd.DataFrame(columns=["imageId","Files"])
ctr = 0

for i in os.listdir(dataset_directory+"/train"):
    train_val_files.loc[ctr,"imageId"] = id[ctr]
    train_val_files.loc[ctr,"Files"] = i
    ctr += 1

train_val_files = train_val_files.sort_values(by="imageId")

train_val_output = pd.merge(left=train_val_age,right=train_val_gender,on="imageId",how="inner")
train_val_output = pd.merge(left=train_val_output,right=train_val_files,on="imageId",how="inner")

train_val_output = train_val_output.sort_values(by="imageId")

train_val_id = train_val_output["imageId"].values.tolist()
np.random.shuffle(train_val_id)


threshold = int(np.round(0.8 * len(train_val_id)))

train_id = train_val_id[0:threshold]
val_id = train_val_id[threshold:]

train_val_output.set_index(keys="imageId",drop=False,inplace=True)

train_output = train_val_output.loc[train_id,:]
val_output = train_val_output.loc[val_id,:]

train_output = train_output.reset_index(drop=True).drop(columns=["imageId"])
val_output = val_output.reset_index(drop=True).drop(columns=["imageId"])

In [4]:
train_output.head()

,age,gender,Files
0,30.0,0,009329.png
1,61.0,0,006252.png
2,28.0,1,003601.png
3,75.0,0,002830.png
4,72.0,0,002254.png


In [5]:
val_output.head()

,age,gender,Files
0,33.0,1,004871.png
1,22.0,0,009768.png
2,29.0,0,003637.png
3,27.0,0,003477.png
4,22.0,1,000806.png


In [6]:
train_images = []
train_data = {}
ctr = 0
for i in tqdm(iterable=train_output["Files"],desc="Processing train data"):
    ages = train_output.loc[ctr,"age"]
    genders = train_output.loc[ctr,"gender"]
    img_org = cv2.imread(dataset_directory + "/train/" + i)
    img_gray = cv2.cvtColor(src=img_org, code=cv2.COLOR_BGR2GRAY)
    img_short = cv2.resize(src=img_gray, dsize=(64,64))
    img = img_short / 255.0
    train_images.append(img)
    ctr += 1

train_images = np.array(train_images)

Processing train data: 100%|██████████| 8562/8562 [05:23<00:00, 26.46it/s]


In [7]:
dataset_directory + "/" + i

'C:/Users/amith/Documents/Datasets/kaggle/kaggle/008017.png'

In [8]:
dataset_directory + "/val/" + i

'C:/Users/amith/Documents/Datasets/kaggle/kaggle/val/008017.png'

In [9]:
val_images = []
val_data = {}
ctr = 0
for i in tqdm(iterable=val_output["Files"],desc="Processing val data"):
    ages = val_output.loc[ctr,"age"]
    genders = val_output.loc[ctr,"gender"]
    img_org = cv2.imread(dataset_directory + "/train/" + i)
    img_gray = cv2.cvtColor(src=img_org, code=cv2.COLOR_BGR2GRAY)
    img_short = cv2.resize(src=img_gray, dsize=(64,64))
    img = img_short / 255.0
    val_images.append(img)
    ctr += 1

val_images = np.array(val_images)

Processing val data: 100%|██████████| 2140/2140 [00:50<00:00, 42.31it/s]


In [10]:
train_images = train_images.reshape(tuple(list(train_images.shape) + [1]))
val_images = val_images.reshape(tuple(list(val_images.shape) + [1]))

In [11]:
val_images.shape

(2140, 64, 64, 1)

In [12]:
model_genders = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64,activation="relu"),
    tf.keras.layers.Dense(units=1,activation="softmax")
])

In [17]:
model_ages = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64,activation="relu"),
    tf.keras.layers.Dense(units=1,activation="relu")
])

In [18]:
train_ages = train_output["age"].values
train_genders = train_output["gender"].values

val_ages = val_output["age"].values
val_genders = val_output["gender"].values

In [19]:
model_genders.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.binary_crossentropy,metrics=['accuracy'])
model_ages.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.mean_absolute_error, metrics=["mae"])

In [20]:
model_genders.fit(x=train_images,y=train_genders,batch_size=32,epochs=10,validation_data=(val_images,val_genders),verbose=1,shuffle=True,callbacks=tf.keras.callbacks.EarlyStopping(patience=2))

Epoch 1/10
268/268 [==============================] - 97s 357ms/step - loss: 0.4174 - accuracy: 0.4191 - val_loss: 0.3289 - val_accuracy: 0.4229
Epoch 2/10
268/268 [==============================] - 40s 151ms/step - loss: 0.3152 - accuracy: 0.4191 - val_loss: 0.3230 - val_accuracy: 0.4229
Epoch 3/10
268/268 [==============================] - 42s 157ms/step - loss: 0.2550 - accuracy: 0.4191 - val_loss: 0.2685 - val_accuracy: 0.4229
Epoch 4/10
268/268 [==============================] - 49s 184ms/step - loss: 0.2099 - accuracy: 0.4191 - val_loss: 0.3768 - val_accuracy: 0.4229
Epoch 5/10
268/268 [==============================] - 44s 164ms/step - loss: 0.1862 - accuracy: 0.4191 - val_loss: 0.2029 - val_accuracy: 0.4229
Epoch 6/10
268/268 [==============================] - 94s 350ms/step - loss: 0.1556 - accuracy: 0.4191 - val_loss: 0.2151 - val_accuracy: 0.4229
Epoch 7/10
268/268 [==============================] - 42s 157ms/step - loss: 0.1256 - accuracy: 0.4191 - val_loss: 0.2120 - val_ac

In [21]:
model_ages.fit(x=train_images,y=train_ages,batch_size=32,epochs=10,validation_data=(val_images,val_ages),verbose=1,shuffle=True,callbacks=tf.keras.callbacks.EarlyStopping(patience=2))

Epoch 1/10
268/268 [==============================] - 124s 460ms/step - loss: 15.4445 - mae: 15.4445 - val_loss: 13.0284 - val_mae: 13.0284
Epoch 2/10
268/268 [==============================] - 45s 167ms/step - loss: 12.0358 - mae: 12.0358 - val_loss: 11.3074 - val_mae: 11.3074
Epoch 3/10
268/268 [==============================] - 99s 371ms/step - loss: 10.8210 - mae: 10.8210 - val_loss: 10.6170 - val_mae: 10.6170
Epoch 4/10
268/268 [==============================] - 134s 498ms/step - loss: 10.4127 - mae: 10.4127 - val_loss: 11.3630 - val_mae: 11.3630
Epoch 5/10
268/268 [==============================] - 41s 152ms/step - loss: 9.8085 - mae: 9.8085 - val_loss: 9.6994 - val_mae: 9.6994
Epoch 6/10
268/268 [==============================] - 43s 161ms/step - loss: 9.3402 - mae: 9.3402 - val_loss: 9.7870 - val_mae: 9.7870
Epoch 7/10
268/268 [==============================] - 42s 158ms/step - loss: 8.9496 - mae: 8.9496 - val_loss: 9.3731 - val_mae: 9.3731
Epoch 8/10
268/268 [=================

In [22]:
tf.keras.models.save_model(model=model_genders,filepath="C:/Users/amith/Documents/Datasets/model_genders.h5",overwrite=True)
tf.keras.models.save_model(model=model_ages,filepath="C:/Users/amith/Documents/Datasets/model_ages.h5",overwrite=True)

C:\Users\amith\AppData\Local\Temp\ipykernel_40836\4190216403.py:1: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model=model_genders,filepath="C:/Users/amith/Documents/Datasets/model_genders.h5",overwrite=True)
C:\Users\amith\AppData\Local\Temp\ipykernel_40836\4190216403.py:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model=model_ages,filepath="C:/Users/amith/Documents/Datasets/model_ages.h5",overwrite=True)


In [23]:
print(dataset_directory + "/test/")

C:/Users/amith/Documents/Datasets/kaggle/kaggle/test/


In [24]:
test_images = []

ctr = 0
for i in tqdm(iterable=sorted(os.listdir(dataset_directory + "/test/")),desc="Processing test data"):
    img_org = cv2.imread(dataset_directory + "/test/" + i)
    img_gray = cv2.cvtColor(src=img_org, code=cv2.COLOR_BGR2GRAY)
    img_short = cv2.resize(src=img_gray, dsize=(64,64))
    img = img_short / 255.0
    test_images.append(img)
    ctr += 1

test_images = np.array(test_images)

Processing test data: 100%|██████████| 11747/11747 [04:19<00:00, 45.24it/s]


In [25]:
test_ages = pd.read_csv("C:\\Users\\amith\\Documents\\Datasets\\sample_submission_age.csv")["age"].copy()
test_genders = pd.read_csv("C:\\Users\\amith\\Documents\\Datasets\\sample_submission_gender.csv")["gender"].copy()

In [26]:
predict_ages = model_ages.predict(test_images)

368/368 [==============================] - 36s 99ms/step


In [29]:
predict_genders = model_genders.predict(test_images)

368/368 [==============================] - 34s 92ms/step


In [32]:
predict_genders

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [43]:
df_pred_age = pd.DataFrame(columns=["imageId","age"])
df_pred_gender = pd.DataFrame(columns=["imageId","gender"])

df_pred_age["imageId"] = list(range(predict_ages.shape[0]))
df_pred_age["age"] = np.int64(np.round(predict_ages,0))

df_pred_gender["imageId"] = list(range(predict_genders.shape[0]))
df_pred_gender["gender"] = np.int64(np.round(predict_genders,0))

In [44]:
df_pred_age

,imageId,age
0,0,48
1,1,32
2,2,36
3,3,60
4,4,35
...,...,...
11742,11742,53
11743,11743,44
11744,11744,29
11745,11745,44


In [45]:
df_pred_gender

,imageId,gender
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
11742,11742,1
11743,11743,1
11744,11744,1
11745,11745,1


In [47]:
df_pred_gender.to_csv(path_or_buf="C:/Users/amith/Documents/Datasets/Saved/sample_submission_gender.csv",index=False)
df_pred_age.to_csv(path_or_buf="C:/Users/amith/Documents/Datasets/Saved/sample_submission_age.csv",index=False)